<a href="https://colab.research.google.com/github/alexjercan/depth-estimation/blob/master/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install matplotlib==3.3.3

In [ ]:
!git clone https://github.com/alexjercan/depth-estimation.git
%cd depth-estimation

import torch
from IPython.display import clear_output

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
# Download bdataset
torch.hub.download_url_to_file('https://github.com/alexjercan/depth-estimation/releases/download/v1.0/bdataset_stereo.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../ && rm tmp.zip

In [ ]:
torch.hub.download_url_to_file('https://github.com/alexjercan/mesh-pose-reconstruction/releases/download/v1.0/normal.pth', 'normal.pth')

In [ ]:
from attr_dict import AttrDict

config_train = AttrDict()

config_train.DATASET_ROOT = "../bdataset_stereo"
config_train.JSON_PATH = "train.json"
config_train.BATCH_SIZE = 2
config_train.IMAGE_SIZE = 256
config_train.WORKERS = 8
config_train.PIN_MEMORY = True
config_train.SHUFFLE = True

config_train.LEARNING_RATE = 1e-3
config_train.BETAS = [.9, .999]
config_train.MILESTONES = [150]
config_train.GAMMA = .5

config_train.NUM_EPOCHS = 10
config_train.OUT_PATH ="./runs"
config_train.LOAD_MODEL = False
config_train.CHECKPOINT_FILE = "normal.pth"

config_test = AttrDict()

config_test.DATASET_ROOT = "../bdataset_stereo"
config_test.JSON_PATH = "test.json"
config_test.BATCH_SIZE = 2
config_test.IMAGE_SIZE = 256
config_test.WORKERS = 8
config_test.PIN_MEMORY = True
config_test.SHUFFLE = False

config_test.OUT_PATH = "./runs"
config_test.LOAD_MODEL = True
config_test.CHECKPOINT_FILE = "normal.pth"

config_detect = AttrDict()

config_detect.JSON = [{"imageL": "data/left.png", "imageR": "data/right.png", "output": "data/depth.exr"}]
config_detect.IMAGE_SIZE = 256
config_detect.CHECKPOINT_FILE = "normal.pth"

In [ ]:
from train import train
train(config_train, config_test)

In [ ]:
from test import test
test(config=config_test)

In [ ]:
from detect import detect
detect(config=config_detect)